In [4]:
def read_input(path):
    res = open(path, "r").readlines()
    return [x.strip() for x in res]


def replace_in_grid(grid, value, replacement):
    for x, l in enumerate(grid):
        for y, c in enumerate(l):
            if c == value:
                grid[x][y] = replacement

    return grid

In [94]:
DIRECTIONS = {"<": (0, -1), "^": (-1, 0), "v": (1, 0), ">": (0, 1)}

NUMERIC = [list(x) for x in ["789", "456", "123", "#0A"]]

DIRECTIONAL = [["#", "^", "A"], ["<", "v", ">"]]


def find_char(grid, char):
    for x, l in enumerate(grid):
        for y, c in enumerate(l):
            if c == char:
                return x, y


def shortest_path(grid, current_char, target_char):
    x, y = find_char(grid, current_char)
    i, j = find_char(grid, target_char)
    gap = find_char(grid, "#")

    if i > x:
        ud = "v" * (i - x)
    else:
        ud = "^" * (x - i)

    if j > y:
        lr = ">" * (j - y)
    else:
        lr = "<" * (y - j)

    if j > y and grid[i][y] != "#":
        return f"{ud}{lr}A"
    elif grid[x][j] != "#":
        return f"{lr}{ud}A"

    return f"{ud}{lr}A"


def input_code(code, grid):
    current_char = "A"
    res = ""
    for c in code:
        res += shortest_path(grid, current_char, c)
        current_char = c
    return res


def input_nested_code(code, nest_count=2):
    seq = input_code(code, NUMERIC)
    for _ in range(nest_count):
        seq = input_code(seq, DIRECTIONAL)
    return seq


def part1(code):
    num = int("".join([x for x in code if x.isdigit()]))
    ans = len(input_nested_code(code))
    return num * ans


sum([part1(x) for x in read_input("input")])

# shortest_path(NUMERIC, "1", "A")
# shortest_path(DIRECTIONAL, "<", "A")
# input_code("123A", NUMERIC)
# input_nested_code("029A")

224326

In [173]:
from functools import cache

numpad_graph = {}
for n1 in [item for row in NUMERIC for item in row]:
    for n2 in [item for row in NUMERIC for item in row]:
        numpad_graph[(n1, n2)] = shortest_path(NUMERIC, n1, n2)


dirpad_graph = {}
for n1 in [item for row in DIRECTIONAL for item in row]:
    for n2 in [item for row in DIRECTIONAL for item in row]:
        dirpad_graph[(n1, n2)] = shortest_path(DIRECTIONAL, n1, n2)


@cache
def get_length(sequence, iterations, first_iter=False):
    if iterations == 0:
        return len(sequence)

    prev = "A"
    total_length = 0
    graph = numpad_graph if first_iter else dirpad_graph
    for char in sequence:
        total_length += get_length(graph[(prev, char)], iterations - 1)
        prev = char
    return total_length


def part2(code):
    num = int("".join([x for x in code if x.isdigit()]))
    ans = get_length(code, 26, True)
    return num * ans

In [175]:
sum([part2(x) for x in read_input("input")])

279638326609472

In [158]:
def increment_transitions(code, nest_count=2):
    directional_transitions = {}
    dirs = [item for row in DIRECTIONAL for item in row]
    for d1 in dirs:
        for d2 in dirs:
            directional_transitions[f"{d1}{d2}"] = (
                shortest_path(DIRECTIONAL, d1, d2),
                0,
            )

    first_directional_seq = input_code(input_code(code, NUMERIC), DIRECTIONAL)

    for i, c in enumerate(first_directional_seq):
        if i == 0:
            prev_c = "A"
        else:
            prev_c = first_directional_seq[i - 1]

        current_seq = f"{prev_c}{c}"
        x = directional_transitions[current_seq]
        directional_transitions[current_seq] = (x[0], x[1] + 1)

    ans = sum(
        [
            len(v[0]) * v[1] * (nest_count - 1)
            for _, v in directional_transitions.items()
        ]
    )
    num = int("".join([x for x in code if x.isdigit()]))

    return ans * num


increment_transitions("208A", 26)
# 17986962866080

364000